# Environment

In [ ]:

! pip install --quiet langchain langchain-openai tiktoken langchainhub chromadb langgraph

In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<LANGCHAIN_API_KEY>'

In [2]:
os.environ['OPENAI_API_KEY'] = '<OPENAI_API_KEY>'

In [3]:
os.environ['TAVILY_API_KEY'] ='<TAVILY_API_KEY>'

## Index

In [4]:

### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# Set embeddings
embd = OpenAIEmbeddings()


In [5]:
# load documents
with open("documents/lyft_10k.txt", "r") as text_file:
    doc = text_file.read()


doc =  Document(page_content=doc, metadata={"source": "local"})

In [6]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents([doc])

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embd,
)

# retrive 12 most relevant documents 
retriever = vectorstore.as_retriever(search_kwargs={"k": 12})


## LLMs

We use a router to pick between tools.

GPT model decides which tool(s) to call, as well as the how to query them.

In [7]:
### Router

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# Preamble
preamble = """You are an expert at routing a user question to a vectorstore or web_search.
The vectorstore contains financial documents related to Lyft, the ride sharing company based out of the US.
Use the vectorstore for questions relating to Lyft, the ride sharing company. For all other questions, use web_search."""

# LLM with function call 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
route_prompt = ChatPromptTemplate.from_messages(
    [   ("system", preamble),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
response = question_router.invoke({"question": "Who will the Bears draft first in the NFL draft?"})
print(response)

response = question_router.invoke({"question": "How will Lyft compete against Uber?"})
print(response)

response = question_router.invoke({"question": "Hi how are you?"})
print(response)

response = question_router.invoke({"question": "What is Lyft's market share in the US?"})
print(response)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


datasource='web_search'
datasource='vectorstore'
datasource='web_search'
datasource='vectorstore'


In [8]:
### Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# Prompt
preamble = """You are a grader assessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

grade_prompt = ChatPromptTemplate.from_messages(
    [     ("system", preamble),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "How will Lyft compete against Uber?"
docs = retriever.invoke(question)
print(len(docs))
for doc in docs:
    print(doc.page_content)
doc_txt = docs[0].page_content
response = retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)
print(doc_txt)

12
Item 1. Business.                                                                
   Overview                                                                         
                                                                                    
       Lyft, Inc. (the “Company” or “Lyft”) started a movement to revolutionize transportation. In 2012, we launched our peer-to-peer marketplace for on-
   demand ridesharing and have continued to pioneer innovations. Today, Lyft is one of the largest multimodal transportation networks in the United States and
   Canada. We have an important purpose, which is to get riders out into the world so they can live their lives together, and to provide drivers a way to work that
   gives them control over their time and money.                                    
       Our ridesharing marketplace connects drivers with riders via the Lyft mobile application (the “Lyft App”) in cities across the United States and in
   select cities in Canada. 

In [9]:
### Generate

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

# Preamble
preamble = """You are an assistant for question-answering tasks based on context documents. Use the following pieces of retrieved context documents to answer the question. If you don't know the answer, just say that you don't know. Use eleven sentences maximum and keep the answer concise."""

# LLM

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# question = "How will Lyft compete against Uber?"
documents = [Document(metadata={}, page_content="The Bears on Thursday, as expected, selected USC quarterback Caleb Williams with the No. 1 pick in the 2024 NFL Draft. Williams was widely considered the top prospect in a draft class loaded with talented quarterbacks.\nHere is how Ryan Poles and Matt Eberflus did with their selections for the Chicago Bears 2024 NFL Draft: Round 1 • Pick 1 (1) • QB Caleb Williams USC. Round 1 • Pick 9 (9) • WR Rome Odunze\nApril 28, 2024 2:38 pm CT. The Chicago Bears selected five players during the 2024 NFL draft, which included landing two of the draft's best players in quarterback Caleb Williams and wide\nThe Chicago Bears made waves during the first round of the 2024 NFL draft, where they added two blue chip players to the roster with their top-10 selections — quarterback Caleb Williams (No. 1\n2023 record: 7-10. Fourth in NFC North; missed playoffs. Bears 2024 draft picks (4): Round 1, pick 1 (from the Panthers in the D.J. Moore/2023 No. 1 overall pick trade) Round 1, pick 9. Round 3")]
question = 'What player are the Bears expected to draft first in the 2024 NFL draft?'
# Prompt
prompt = ChatPromptTemplate.from_messages(
        [ ("system", preamble),
            # HumanMessage(
            #     f"Question: {x['question']} \nAnswer: ",
            #     additional_kwargs={"documents": x["documents"]},
            # )
            ("human", "Answer the following question {question} based on these context documents:{documents}"),
        ]
    )


# Chain
rag_chain = prompt | llm | StrOutputParser()



# Run
generation = rag_chain.invoke({"documents": documents, "question": question})
print(generation)

The Bears are expected to draft USC quarterback Caleb Williams first in the 2024 NFL Draft. He was selected with the No. 1 pick, as he was widely regarded as the top prospect in a strong quarterback class. Additionally, the Bears also picked wide receiver Rome Odunze with their ninth pick in the first round.


In [10]:
### LLM fallback

from langchain_core.output_parsers import StrOutputParser

# Preamble
preamble = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. Use seven sentences maximum and keep the answer concise."""

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
def prompt(x):
    return ChatPromptTemplate.from_messages(
        
        [("system", preamble),
            HumanMessage(f"Question: {x['question']} \nAnswer: ")]
    )


# Chain
llm_chain = prompt | llm | StrOutputParser()

# Run
question = "Hi how are you? Prove that 1 plus 1 is 2."
generation = llm_chain.invoke({"question": question})
print(generation)

Hello! To prove that 1 plus 1 equals 2, we can use the Peano axioms, which are a set of axioms for the natural numbers. According to these axioms, we define the number 1 as the successor of 0, denoted as S(0). The number 2 is defined as the successor of 1, or S(1). 

When we add 1 to 1, we are essentially finding the successor of 1, which is S(1) = 2. Therefore, 1 + 1 = S(0) + S(0) = S(1) = 2. This formalizes the intuitive understanding that combining one unit with another unit results in two units.


In [11]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# Preamble
system = """You are a grader assessing whether an LLM generation is supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'yes' means that the answer is supported by the set of facts. Otherwise, 'no'."""

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)


# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})



GradeDocuments(binary_score='no')

In [12]:
### Answer Grader
from typing import Literal

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: Literal["yes", "no"] = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# Preamble
preamble = """You are a grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no'. Yes means that the answer resolves the question."""



#LLM with function call 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(GradeAnswer)

#Prompt 
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", preamble),
         ("human", "User question: \n\n {question} \n\n LLM answer: {generation}"),
    ]
)


question = 'What player are the Bears expected to draft first in the 2024 NFL draft?'
generation = ('The Chicago Bears are expected to draft USC quarterback Caleb Williams with ',
 'the first pick in the 2024 NFL draft. Williams was considered the top ',
 'prospect in a draft class loaded with talented quarterbacks.')

answer_grader = answer_prompt | structured_llm_grader
score = answer_grader.invoke({"question": question, "generation": generation})
if score != None:
    print(score)
else:
    print(score)




binary_score='yes'


In [13]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

## Graph
Capture the flow in as a graph.

### Graph state

In [14]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """|
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

### Graph Flow

In [15]:
from langchain.schema import Document
import pprint


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def llm_fallback(state):
    """
    Generate answer using the LLM w/o vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---LLM Fallback---")
    question = state["question"]
    generation = llm_chain.invoke({"question": question})
    return {"question": question, "generation": generation}


def generate(state):
    """
    Generate answer using the vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    if not isinstance(documents, list):
        documents = [documents]

    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]


    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})

    # Fallback to LLM or raise error if no decision
    if source is None or source.datasource == None:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"

    # Choose datasource
    datasource = source.datasource
    if datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    else:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score
    print("documents", documents)
    print("generation",generation)

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = 'none' if score is None else score.binary_score
        # pprint.pprint(question)
        # pprint.pprint(generation)
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [16]:

from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # rag
workflow.add_node("llm_fallback", llm_fallback)  # llm

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
        "llm_fallback": "llm_fallback",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",  # Hallucinations: re-generate
        "not useful": "web_search",  # Fails to answer question: fall-back to web-search
        "useful": END,
    },
)
workflow.add_edge("llm_fallback", END)

# Compile
app = workflow.compile()

In [17]:
def generate_answer(question: str):
    # Run
    inputs = {
        "question": question
    }
    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint.pprint(f"Node '{key}':")
            # Optional: print full state at each node
        pprint.pprint("\n---\n")

    # Final generation
    pprint.pprint(value["generation"])   
    return value["generation"]

In [18]:
generate_answer("What player are the Bears expected to draft first in the 2024 NFL draft?")

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
documents [Document(metadata={}, page_content="Bears 2024 NFL Draft First-Round Prediction. Pick: No. 1 Overall. Player Prediction: Caleb Williams - QB, USC DRAFT ANALYSIS: It honestly feels like Caleb Williams' name has been penciled in this\nThe Bears on Thursday, as expected, selected USC quarterback Caleb Williams with the No. 1 pick in the 2024 NFL Draft. Williams was widely considered the top prospect in a draft class loaded with talented quarterbacks.\nThe 2024 NFL Draft is OVAH! All 257 picks are in the books. Here are the picks made by Chicago Bears General Manager Ryan Poles and the rest of his staff: Bears Draft Picks Night 1. Round 1, Pick 1: Caleb Williams, Southern California quarterback. QUICK HIT: I'm not even sure what is left to write about Caleb Williams at this point.\nThe Chicago Bears selected five players during the 202

'The Chicago Bears are expected to draft Caleb Williams, a quarterback from USC, with the first overall pick in the 2024 NFL Draft. Williams is widely regarded as the top prospect in a draft class rich with talented quarterbacks. The Bears have retained the No. 1 pick for the upcoming draft, which is scheduled to take place on April 25, 2024. This selection aligns with predictions made prior to the draft, indicating a strong consensus around Williams as the choice for Chicago.'

In [19]:
# Run
generate_answer("How will Lyft compete against Uber?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
documents [Document(metadata={'source': 'local'}, page_content='Item 1. Business.                                                                \n   Overview                                                                         \n                                                                                    \n       L

"Lyft plans to compete against Uber by focusing on several strategic initiatives. First, they aim to increase rider use cases by expanding their service offerings, such as the Lyft Pink subscription plan and first-mile and last-mile services. They are also working to grow their active rider base through brand investments and growth marketing to enhance consumer preference for Lyft. \n\nAdditionally, Lyft is committed to providing a differentiated rideshare experience, which includes features like Women+ Connect to cater to specific demographics. They are prioritizing competitive service levels to attract and retain more drivers and riders, which is crucial for maintaining their market position. \n\nLyft is also looking to grow its share of consumers' transportation spending by addressing a wide range of mobility needs, including bike and scooter rentals. They recognize the competitive landscape and are prepared to react to market changes, even if it means making short-term sacrifices l

In [21]:
# Run
generate_answer( "What is Lyft's market share in the US?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---
"Node 'grade_documents':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
documents [Document(metadata={}, page_content="Lyft's Market Position: Lyft is the second-largest ride-hailing platform in the US, with a significant market share of 32%. It is particularly dominant in Phoenix, Arizon

"Lyft's market share in the US is reported to be 32%. However, data from Bloomberg Second Measure indicates that it had a market share of 25% in May 2023. Additionally, another source mentions that Lyft's market share is 29%. This suggests some variability in reported figures, but the most consistent figure appears to be around 32%. Lyft is the second-largest ride-hailing platform in the US, particularly strong in cities like Phoenix and Detroit, where it holds a 45% market share. Overall, while the exact percentage may vary slightly depending on the source, Lyft's market share is generally recognized to be around 25% to 32%."

In [22]:
# Run
generate_answer("What is the biggest risk for Lyft?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
documents [Document(metadata={'source': 'local'}, page_content="• the unpredictability of our results of operations and uncertainty regarding the growth of the ridesharing and other markets;\n     • our ability to attract and retain qualified drivers and riders;              \n                                                 

"The biggest risk for Lyft appears to be the unpredictability of its operational results and the uncertainty regarding the growth of the ridesharing market. This includes challenges in attracting and retaining qualified drivers and riders, which directly impacts ride requests and overall business performance. Additionally, Lyft faces significant financial risks related to its insurance coverage and the adequacy of reserves for auto-related claims. The company's reliance on third-party services, such as payment processors and cloud services, also poses operational risks. \n\nMoreover, Lyft's ability to manage its pricing methodologies and offer competitive ride pricing is crucial for retaining riders. Any disruptions to its platform, such as data security breaches or service outages, could severely affect user trust and engagement. Legal challenges and regulatory investigations further complicate the operational landscape. Lastly, maintaining a positive brand reputation and effectively 

In [23]:
# Run
generate_answer("In what areas is Lyft looking to innovate in the future?")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
documents [Document(metadata={'source': 'local'}, page_content='plans to:                                                                        \n     • Increase Rider Use Cases. We are continuously working to make Lyft the transportation network of choice across an expanding range of use cases. We\n       offer products to simplify

"Lyft is looking to innovate in several key areas for the future. They aim to increase rider use cases by expanding their transportation network to cover a broader range of travel needs, including subscription plans and commuter programs. Lyft plans to grow its active rider base through brand investments and marketing strategies, while also enhancing the variety of ride experiences offered, such as Wait & Save and Priority Pickup. \n\nAdditionally, Lyft is focused on growing its share of consumers' transportation spending by addressing diverse mobility needs and expanding its network coverage. They are also committed to improving the ridesharing marketplace's efficiency through technology investments, including advancements in payments and data science. \n\nSustainability is another area of innovation, with efforts to transition rides from gas-powered vehicles to electric vehicles (EVs) and promote environmentally friendly options like the Green program. Lyft is also enhancing the driv